In [ ]:
import os
import sys
import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import cv2
import gc
import seaborn as sns
import albumentations as A
import matplotlib.pyplot as plt

import torch
import torch.nn as nn


sys.path.append("../input/efficientnet")
from  efficientnet_pytorch import EfficientNet

training kernel : https://www.kaggle.com/narendra/pawpularity-baseline-training/data?scriptVersionId=81509940

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu' )
print(device)

# model

In [ ]:
class Baseline(nn.Module):
    def __init__(self):
        super(Baseline, self).__init__()
        
        self.efficient_net=EfficientNet.from_pretrained('efficientnet-b0', include_top=True)
        self.avg_pooling=nn.AdaptiveAvgPool2d(1)
        
        self.fc=nn.Sequential(
            nn.BatchNorm1d(1280),
            nn.Linear(1280, 512),
            nn.SiLU(),
            nn.Dropout(0.2),
            
            nn.BatchNorm1d(512),
            nn.Linear(512, 1)
        )
    def forward(self, x):
        batch_size=x.size(0)
        x=self.efficient_net.extract_features(x)
        x=self.avg_pooling(x).view(batch_size, -1)
        x=self.fc(x)
        x=torch.sigmoid(x)
        return x

# read image

In [ ]:
test_folder="../input/petfinder-pawpularity-score/test"
val_transform = A.Compose([A.Normalize(p=1.0)])

def read_image(image_name):
    filepath=os.path.join(test_folder, image_name)
    img=cv2.imread(filepath)
    img=cv2.resize(img, (256, 256))
    img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img=val_transform(image = img)['image']
    return img

# dataset

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, image_ids):
        self.image_ids=image_ids
    def __getitem__(self, idx):
        image_name = self.image_ids[idx]
        img=read_image(image_name)
        X=torch.tensor(img, dtype=torch.float32).transpose(0, 2)
        return X
        
    def __len__(self):
        return len(self.image_ids)

In [ ]:
BATCH_SIZE=128

test_images=os.listdir(test_folder)
test_dataset=Dataset(test_images)

test_dataloader=torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE,
                                            shuffle=False,
                                            drop_last=False)



In [ ]:
model=torch.load('../input/pawpularity-baseline/model1.pth', map_location=device)
model.eval()

ypred=[]
for X in test_dataloader:
    X=X.to(device)
    with torch.no_grad():
        yhat=model(X).view(-1).cpu().tolist()
        ypred+=yhat

ypred=np.array(ypred)*100

In [ ]:
submission_df=pd.DataFrame.from_dict({
    'Id': test_images,
    'Pawpularity': ypred
})

submission_df['Id'] = submission_df['Id'].apply(lambda x: x.replace('.jpg', ''))
submission_df.head()

In [ ]:
submission_df.to_csv("submission.csv", index=False)